# Part 2 - Network load evaluation


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

from package import constants
from package.connectivity import bfs_is_connected
# from networkx import erdos_renyi_graph, random_regular_graph, adjacency_matrix
import networkx as nx

N = 10000

## 2.1

In [ ]:
k = constants.n_ports//2
# create core random routers graph
rrg = nx.random_regular_graph(d = k, n=N//k+1)

# add endpoints
for n in list(rrg.nodes()):
    endnodes = range(-n*k-1,-(n+1)*k-1,-1)
    rrg.add_nodes_from(endnodes, server=True)
    rrg.add_edges_from(zip([n]*k,endnodes))
rrg_servers = leaves(rrg)

# when considering only the number of jumps between two different servers, a fat-tree is equivalent to a tree
ft = nx.full_rary_tree(r = k, n=(k*N - 1)//(k-1) + 1)


In [ ]:
def leaves(G: nx.Graph) -> list:
    '''Counts the number of leaves in an undirected graph'''
    leaves = [x for x in G.nodes() if G.degree(x)==1]
    return leaves

ft_servers = leaves(ft)
ft.add_nodes_from(ft, server=True)


In [ ]:
def n_closest(g: nx.Graph, src: int, n: int, targets: set = None) -> list :
    sp = nx.single_source_shortest_path_length(rrg, src)
    del sp[src]
    servers = {k for k,v in rrg.nodes(data=True) if 'server' in v} if targets is None else targets
    sp_servers = {k:sp[k] for k in sp.keys() & servers}
    
    return dict(sorted(sp_servers.items(), key=lambda item: item[1]))

n_closest(rrg, -1, 10)

In [ ]:
servers = {k for k,v in rrg.nodes(data=True) if 'type' in v} 
servers